In [1]:
import pandas as pd 

In [3]:
train = pd.read_csv('/content/train.csv')
X_test = pd.read_csv('/content/test.csv')

train.fillna(0, inplace = True)
X_test.fillna(0 , inplace= True)


In [4]:
from sklearn. preprocessing import OneHotEncoder

y_train = train['Transported']
X_train = train.drop(columns = ['Transported'],axis = 1)


name = X_train['Name']
X_train = X_train.drop( columns = ['Name'] , axis = 1 )


In [5]:

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [6]:

name = name.astype('str')

In [7]:
name = labelencoder.fit_transform(name)

In [8]:
name_df = pd.DataFrame(name)
#rename

In [9]:
passengers = []
ids = []

Pass_id_dict = {}
for i in X_train.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1])
  

 

#X_train.loc[X_train['PassengerId']. isin(Pass_id_dict.keys())]
  
X_train['Group'] = passengers
X_train['Number_in_group'] = ids


In [ ]:
X_train = X_train.drop( columns = ['PassengerId'] , axis = 1 )


In [10]:
import numpy as np

deck =[]
num=[]
side=[]

for i in X_train.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_train['Deck'] = deck
X_train['Num'] = num
X_train['Side'] = side
X_train = X_train.drop( columns = ['Cabin' ] , axis = 1 )

categorical = X_train[['HomePlanet' , 'Destination'  , 'Deck' , 'Side' ]]
categorical = pd.get_dummies(categorical)




numerical = X_train[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]



In [11]:
categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)


In [12]:
numerical = numerical.astype('int')

X_train_f = categorical.join(numerical)

In [13]:
X_train_f = X_train_f.join(name_df)
X_train_f = X_train_f.rename( columns = { 0 : 'Name'  })

In [14]:


y_train = y_train.astype('int')



In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()

names = X_train_f.columns 
X_train = scaler.fit_transform(X_train_f)

X_train = pd.DataFrame( X_train , columns=names)
 

In [17]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.7 MB/s eta 0:00:00


In [85]:
#Ансамбль
# https://towardsdatascience.com/a-practical-guide-to-stacking-using-scikit-learn-91e8d021863d
#! pip install catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier


base_models = [
               ('rf',  RandomForestClassifier(  n_estimators = 100 , criterion = "entropy") ) , 
               ( 'KNN' , KNeighborsClassifier( algorithm = 'ball_tree') ) , 
               ( 'SVC' , SVC() ) ,
               ( 'Bayes' , BernoulliNB() ) , 
               ( 'DecisionTree' , DecisionTreeClassifier() ) , 
               ( 'Xgboost' , XGBClassifier(objective="binary:logistic") ) 
               

#LogReg
               ]
meta_model = CatBoostClassifier( 
    eval_metric = 'Accuracy' , loss_function = 'Logloss' ,
    learning_rate = 0.029999999329447746 ,
    iterations = 200 ,
    custom_metric = 'Accuracy' ,
    verbose = 10 ,
    l2_leaf_reg =3                   )

stacking_model = StackingClassifier(estimators=base_models, 
                                    final_estimator=meta_model, 
                                    passthrough= True , 
                                    cv=5,
                                    verbose=2,
                                    stack_method = 'predict')


# params = {
    
#   'rf__bootstrap' : [True , False ] , 
#   'rf__n_estimators' : [ 100 ,  150 ,  200]  ,
#   'rf__criterion' : ['gini' , 'entropy'  , 'log_loss'] 
  

# }

# grid = GridSearchCV(estimator= stacking_model, param_grid=params, cv=2 , scoring = 'accuracy' , verbose = 3)




In [86]:
stacking_model.fit(X_train , y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   13.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

0:	learn: 0.7952375	total: 6.59ms	remaining: 1.31s
10:	learn: 0.8034050	total: 60.4ms	remaining: 1.04s
20:	learn: 0.8043253	total: 112ms	remaining: 951ms
30:	learn: 0.8049005	total: 165ms	remaining: 897ms
40:	learn: 0.8057057	total: 224ms	remaining: 868ms
50:	learn: 0.8067411	total: 276ms	remaining: 807ms
60:	learn: 0.8072012	total: 328ms	remaining: 749ms
70:	learn: 0.8085816	total: 381ms	remaining: 692ms
80:	learn: 0.8098470	total: 439ms	remaining: 644ms
90:	learn: 0.8108823	total: 494ms	remaining: 591ms
100:	learn: 0.8115725	total: 553ms	remaining: 542ms
110:	learn: 0.8130680	total: 605ms	remaining: 485ms
120:	learn: 0.8149085	total: 659ms	remaining: 430ms
130:	learn: 0.8162890	total: 714ms	remaining: 376ms
140:	learn: 0.8174393	total: 767ms	remaining: 321ms
150:	learn: 0.8185897	total: 822ms	remaining: 267ms
160:	learn: 0.8206603	total: 878ms	remaining: 213ms
170:	learn: 0.8223858	total: 936ms	remaining: 159ms
180:	learn: 0.8238813	total: 990ms	remaining: 104ms
190:	learn: 0.8262970

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(criterion='entropy')),
                               ('KNN',
                                KNeighborsClassifier(algorithm='ball_tree')),
                               ('SVC', SVC()), ('Bayes', BernoulliNB()),
                               ('DecisionTree', DecisionTreeClassifier()),
                               ('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))],
                   final_estimator=<catboost.core.CatBoostClassifier object at 0x7faef16838e0>,
                   passthrough=True, stack_method='predict', verbose=2)

In [21]:

passenger_id = X_test['PassengerId'] # оставить на этом месте 

In [22]:
# test
name = X_test['Name']
X_test = X_test.drop(columns =['Name'] , axis = 1 )

labelencoder = LabelEncoder()
name = name.astype('str')
name = labelencoder.fit_transform(name)
name_df = pd.DataFrame(name)

passengers = []
ids = []
Pass_id_dict = {}
for i in X_test.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1]) 
X_test['Group'] = passengers
X_test['Number_in_group'] = ids

X_test = X_test.drop( columns = ['PassengerId'] , axis = 1 )
X_test.head(20)


import numpy as np
deck =[]
num=[]
side=[]
for i in X_test.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_test['Deck'] = deck
X_test['Num'] = num
X_test['Side'] = side
X_test = X_test.drop( columns = ['Cabin' ] , axis = 1 )

categorical = X_test[['HomePlanet' , 'Destination'  , 'Deck' , 'Side']]
categorical = pd.get_dummies(categorical)




numerical = X_test[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]
categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)
numerical = numerical.astype('int')

X_test_f = categorical.join(numerical)

X_test_f = X_test_f.join(name_df)
X_test_f = X_test_f.rename( columns = { 0 : 'Name'  })
X_test_f

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,13,1,3,2913
1,1,0,0,0,0,1,0,0,0,0,...,0,0,9,0,2823,0,18,1,4,2407
2,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,19,1,0,3377
3,0,1,0,0,0,1,0,0,1,0,...,0,0,6652,0,181,585,21,1,1,2712
4,1,0,0,0,0,1,0,0,0,0,...,0,10,0,635,0,0,23,1,5,669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,9266,2,1496,2106
4273,1,0,0,0,0,1,0,0,0,0,...,0,0,847,17,10,144,9269,1,0,2655
4274,0,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,9271,1,296,2086
4275,0,1,0,0,0,0,0,0,0,1,...,0,0,2680,0,0,523,9273,1,297,2328


In [23]:


scaler = StandardScaler()

names = X_test_f.columns 
X_test = scaler.fit_transform(X_test_f)

X_test = pd.DataFrame( X_test , columns=names)



In [24]:
X_test


,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.703425,-0.489847,-1.146527,0.708477
1,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.701584,-0.489847,-1.144593,0.297314
2,-1.060016,1.807889,-0.525314,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.701215,-0.489847,-1.152328,1.085511
3,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.700479,-0.489847,-1.150395,0.545149
4,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.699743,-0.489847,-1.142660,-1.114938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.703574,0.492373,1.740519,0.052730
4273,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,0.277095,-0.281538,-0.258790,-0.130193,1.704679,-0.489847,-1.152328,0.498833
4274,-1.060016,-0.553131,1.903624,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.705415,-0.489847,-0.579947,0.036478
4275,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,1.491019,-0.312173,-0.267841,0.176479,1.706152,-0.489847,-0.578013,0.233121


In [87]:
y_test = stacking_model.predict(X_test)

In [88]:
result = pd.DataFrame({'PassengerId' : passenger_id , 'Transported' : y_test.astype(bool)})

In [89]:
result

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [90]:
result.to_csv ('/content/result_algorythm_balltree.csv', index= False )

In [39]:
X_train

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,3.187347,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,-1.734409,-0.491161,-1.141624,0.446353
1,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.168073,-0.275387,-0.241771,0.217158,-0.224205,-1.734034,-0.491161,-1.141624,0.146306
2,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,5.740821,-0.313741,-0.30661,-0.241218,...,6.533255,-0.268001,1.959998,-0.283579,5.695623,-0.219796,-1.733660,-0.491161,-1.141624,-1.471948
3,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,5.740821,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,0.523010,0.336851,2.687176,-0.092818,-1.733660,0.457443,-1.141624,1.205272
4,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,0.125652,-0.237159,-0.031059,0.231374,-0.261240,-1.733286,-0.491161,-1.139678,1.673345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-1.060617,1.754795,-0.503664,1.956897,-0.317487,-1.459188,5.740821,-0.313741,-0.30661,-0.241218,...,6.533255,-0.333105,3.992336,-0.283579,1.189173,-0.197751,1.738236,-0.491161,-0.950907,-0.244955
8689,0.942847,-0.569867,-0.503664,-0.511013,3.149739,-1.459188,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,1.738984,-0.491161,1.775567,0.257523
8690,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,2.846999,-0.269737,-0.263003,1.739359,-0.491161,1.777513,-0.453788
8691,-1.060617,1.754795,-0.503664,1.956897,-0.317487,-1.459188,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,0.376365,-0.283579,0.043013,2.589576,1.739733,-0.491161,0.041600,-1.016276


In [ ]:
X_test

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.703425,-0.489847,-1.146527,0.708477
1,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.701584,-0.489847,-1.144593,0.297314
2,-1.060016,1.807889,-0.525314,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.701215,-0.489847,-1.152328,1.085511
3,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.700479,-0.489847,-1.150395,0.545149
4,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.699743,-0.489847,-1.142660,-1.114938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.703574,0.492373,1.740519,0.052730
4273,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,0.277095,-0.281538,-0.258790,-0.130193,1.704679,-0.489847,-1.152328,0.498833
4274,-1.060016,-0.553131,1.903624,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.705415,-0.489847,-0.579947,0.036478
4275,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,1.491019,-0.312173,-0.267841,0.176479,1.706152,-0.489847,-0.578013,0.233121
